# Hyper parameter tuning

What can you tweak in neural network:

1. Number of layers
2. Number of Neurons
3. Type of activation function

In [15]:
import numpy as np

from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal


In [12]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_new = X_test[:3]

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))

    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    
    return model

In [13]:
# Wrapper for using the Scikit-Learn API with Keras models.
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg.fit(
    X_train, 
    y_train, 
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100


/var/folders/8t/3rnys97s4mj60s_wk489wkp80000gn/T/ipykernel_16318/3313595958.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
/Users/soomin/.pyenv/versions/miniforge3-4.10.1-5/envs/handson-conda/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 0s 487us/step - loss: 1.2378 - val_loss: 0.6579
Epoch 2/100
363/363 [==============================] - 0s 399us/step - loss: 0.6133 - val_loss: 0.5629
Epoch 3/100
363/363 [==============================] - 0s 400us/step - loss: 0.5496 - val_loss: 0.5111
Epoch 4/100
363/363 [==============================] - 0s 397us/step - loss: 0.5140 - val_loss: 0.4906
Epoch 5/100
363/363 [==============================] - 0s 398us/step - loss: 0.4923 - val_loss: 0.4663
Epoch 6/100
363/363 [==============================] - 0s 388us/step - loss: 0.4752 - val_loss: 0.4498
Epoch 7/100
363/363 [==============================] - 0s 395us/step - loss: 0.4622 - val_loss: 0.4375
Epoch 8/100
363/363 [==============================] - 0s 396us/step - loss: 0.4546 - val_loss: 0.4274
Epoch 9/100
363/363 [==============================] - 0s 391us/step - loss: 0.4456 - val_loss: 0.4233
Epoch 10/100
363/363 [==============================] - 0s 395us/step - loss: 0.4405 

In [16]:
param_distribs = {
"n_hidden": [0, 1, 2, 3],
"n_neurons": np.arange(1, 100),
"learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(
    X_train, 
    y_train, 
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/100
164/242 [===================>..........] - ETA: 0s - loss: 5.3843 

/Users/soomin/.pyenv/versions/miniforge3-4.10.1-5/envs/handson-conda/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 0s 576us/step - loss: 4.6123 - val_loss: 2.6273
Epoch 2/100
242/242 [==============================] - 0s 423us/step - loss: 1.6637 - val_loss: 1.1680
Epoch 3/100
242/242 [==============================] - 0s 419us/step - loss: 0.8781 - val_loss: 0.7492
Epoch 4/100
242/242 [==============================] - 0s 420us/step - loss: 0.6557 - val_loss: 0.6237
Epoch 5/100
242/242 [==============================] - 0s 417us/step - loss: 0.5891 - val_loss: 0.5797
Epoch 6/100
242/242 [==============================] - 0s 417us/step - loss: 0.5671 - val_loss: 0.5634
Epoch 7/100
242/242 [==============================] - 0s 428us/step - loss: 0.5574 - val_loss: 0.5532
Epoch 8/100
242/242 [==============================] - 0s 423us/step - loss: 0.5519 - val_loss: 0.5507
Epoch 9/100
242/242 [==============================] - 0s 418us/step - loss: 0.5476 - val_loss: 0.5464
Epoch 10/100
242/242 [==============================] - 0s 417us/step - loss: 0.5438 

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x136cfb580>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1376a01f0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [18]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model = rnd_search_cv.best_estimator_.model

{'learning_rate': 0.010243512177571289, 'n_hidden': 3, 'n_neurons': 12}
-0.32605581482251483


# Rule of thumbs for 
1. Number of hidden layers
2. Number of Neurons
3. Learning rates, batch size and other hyperparameters

1. Number of hidden layers:
    For most of the simple problem, it is sufficient to have one or two hidden layers
    For more complex problem, more layers are needed. You can try to increase the number of layers up until it starts to overfit.
    For a more complex problem such as image recognition and speech recognition, you rarely have to train from scratch. It is much more common to reuse the part of pretrained part of the pre-trained state of the art network.

2. Number of Neurons per hidden layer:
    The number of Neurons for the input and output layer is decided by input data's shape and type of the task.
    It used to be common to size the number of neurons in hidden layers in pyramid shape. However, this practice has been abandoned because the same number of neurons per hidden layer seems to perform as good as having a pyramid shape. And this has an advantage that you have only one hyperparameter to tune rather than each per hidden layer.
    Like the number of layers, try to increase the number of neurons gradually till it overfits the data. It is more simple and efficient to pick a model with more layers and more neurons than you actually need, and then use early stopping and other regularization techniques to prevent it from overfitting.

In general, you get more bang for buck by increasing the number of layers than increasing the number of neurons per hidden layer.

3. Learning rate, batch size, and other hyperparameters:
- Learning rate: arguably the most important hyperparmeters
In general, optimal learning rate is about half of the maximum learning rate.

- Learning rate:
    - train a model for a few hundred iterations starting with a very small learning rate and increasing it up to a very large value. Plot the loss function x learning rate 
    - the optimal learning rate is a bit lower than the point at which the loss starts to climb up. 

- Optimizer:
    - Choose a better optimizer than plain old mini-batch gradient descent.
    
- Batch size:
    - Try use a large batch size, use learning rate warm up, if training is unstable, then try using a small batch size. Literature shows contradictory results.
    - Benefit of Large batch size : GPU can process efficiently
    - Caveat of Large batch size : training instability especially in the beginning

    - Benefit of Small batch size : generalize better than the model with large batch size
    - Caveat of Small batch size : takes longer time to train

- Activation funciton:
    - Relu activation function is a good default. The activation function for the output layer depends on the task


- Number of iterations:
    - Number of iterations usually don't matter, just use early stopping instead

** Learning rate and batch size are correlated. Thus, make sure to tweak both of them together.
